# Phiên 1 – Khởi động Chat (Foundry Local)

Notebook này khởi động Foundry Local, tải xuống bí danh mô hình ưu tiên, và thực hiện cả hoàn thành chat tiêu chuẩn và hoàn thành chat theo luồng.


# Kịch bản
Phiên này giới thiệu những điều cơ bản nhất để làm cho một mô hình ngôn ngữ nhỏ hoạt động cục bộ thông qua Foundry Local. Bạn sẽ:
- Cài đặt SDK / các phụ thuộc của client.
- Khởi tạo trình quản lý Foundry Local cho một alias đã chọn (mặc định: `phi-3.5-mini`).
- Áp dụng một bản vá phòng thủ để xử lý các trường tùy chọn trong metadata của mô hình.
- Gửi một yêu cầu hoàn thành trò chuyện tiêu chuẩn.
- Phát trực tiếp phản hồi từng token một.

Mục tiêu là xác nhận runtime cục bộ và đường dẫn mạng của bạn trước khi chuyển sang RAG, định tuyến, hoặc các agent.


### Giải thích: Cài đặt phụ thuộc
Cài đặt các gói Python cần thiết cho luồng trò chuyện tối giản này:
- `foundry-local-sdk`: Quản lý các mô hình cục bộ và vòng đời dịch vụ.
- `openai`: Abstraction client quen thuộc cho các hoàn thành trò chuyện.
- `rich`: In đẹp để hiển thị rõ ràng hơn trong notebook.

Việc chạy lại là an toàn (idempotent). Bỏ qua nếu môi trường của bạn đã có các gói này.


In [1]:
# Install required libraries (idempotent)
!pip install -q foundry-local-sdk openai rich

### Giải thích: Các Thư Viện Cốt Lõi
Đưa vào các module được sử dụng xuyên suốt notebook:
- `FoundryLocalManager` để tương tác với môi trường runtime của mô hình cục bộ.
- Client `OpenAI` để chúng ta có thể tái sử dụng giao diện API hoàn thành chat quen thuộc.
- `rich.print` để xuất kết quả có định dạng.

Không có cuộc gọi mạng nào diễn ra ở đây—chỉ là chuẩn bị không gian tên.


In [2]:
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI
from rich import print

### Giải thích: Khởi tạo Manager & Sửa đổi Metadata
Khởi tạo `FoundryLocalManager` cho bí danh đã chọn và áp dụng một bản vá phòng thủ để xử lý linh hoạt các phản hồi dịch vụ khi `promptTemplate` có thể là `null`.

Kết quả chính:
- Xác nhận trạng thái dịch vụ và điểm cuối.
- Liệt kê các mô hình được lưu trong bộ nhớ cache (xác minh kho lưu trữ cục bộ).
- Giải quyết ID mô hình cụ thể cho bí danh (được sử dụng trong các cuộc gọi chat sau này).

Nếu bạn gặp vấn đề xác thực trong metadata dịch vụ thô, mẫu này cho thấy cách làm sạch mà không cần chỉnh sửa SDK.


In [3]:
# Catalog-safe manager initialization (handles null promptTemplate values)
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-3.5-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:57127/v1

Cached models:
[
    FoundryModelInfo(
        alias='gpt-oss-20b',
        id='gpt-oss-20b-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/gpt-oss-20b-cuda-gpu/versions/1',
        file_size_mb=9882,
        prompt_template={},
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-cuda-gpu/versions/1',
        file_size_mb=2181,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-cuda-gpu:4',
        version='4',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-cuda-gpu/versions/4',
        file_size_mb=3686,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-cuda-gpu/versions/3',
        file_size_mb=528,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-7b',
        id='qwen2.5-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-coder-7b',
        id='qwen2.5-coder-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-coder-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_star

Using model id: Phi-4-mini-instruct-cuda-gpu:4

### Giải thích: Hoàn thành Chat cơ bản
Tạo một client tương thích với `OpenAI` hướng đến điểm cuối Foundry cục bộ và thực hiện một lần hoàn thành chat không streaming. Tập trung vào:
- Đảm bảo mô hình phản hồi mà không gặp lỗi.
- Xác thực độ trễ / định dạng đầu ra.
- Giữ `max_tokens` ở mức vừa phải để tiết kiệm tài nguyên.

Nếu điều này thất bại, hãy kiểm tra lại rằng dịch vụ Foundry Local đang chạy và alias được giải quyết chính xác.


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':prompt}], max_tokens=120, temperature=0.5)
print(resp.choices[0].message.content)

1. Enhanced Data Privacy: Local inference allows data to be processed locally on a user's device rather than being 
sent to a central server. This reduces the risk of sensitive information being intercepted or exposed during 
transmission, thereby enhancing privacy.

2. Reduced Data Collection: By performing inference locally, the need for extensive data collection is minimized. 
This means that less personal information needs to be stored or transmitted, further protecting user privacy.

### Giải thích: Hoàn thành trò chuyện theo luồng
Minh họa việc truyền tải token theo luồng để cải thiện độ trễ cảm nhận và trải nghiệm người dùng tương tác. Vòng lặp in các thay đổi gia tăng khi chúng đến:
- Hữu ích cho giao diện trò chuyện nơi đầu ra một phần sớm có ý nghĩa.
- Cho phép bạn đo lường tốc độ truyền tải token so với độ trễ hoàn thành toàn bộ.

Bạn có thể điều chỉnh mẫu này để tích lũy token, cập nhật tiện ích tiến độ, hoặc hủy giữa chừng quá trình tạo.


In [5]:
# Streaming example
stream = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}], stream=True, max_tokens=60, temperature=0.4)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

with

a

central

server

.


---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
